In [1]:
%load_ext autoreload
%autoreload 2

## Imports

In [2]:
import sys
import os
import pickle
import random

import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt

from torch.optim import Adam
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from collections import OrderedDict 

sys.path.append(os.path.abspath('..'))

import datasets
import utils.more_torch_functions as mtf
from utils.modules import Parallel, MaxLayer
from utils.custom_activations import StepActivation
from utils.custom_loss import AsymBCELoss
from utils.misc import train_model
from utils.test import Model

seed = 2872
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

SAVE_PATH = os.path.join(os.path.abspath('..'), "backup")
PKL_PATH = os.path.join(SAVE_PATH, "bdd")
PTH_PATH = os.path.join(SAVE_PATH, "nn")
METRIC_PATH = os.path.join(SAVE_PATH, "metrics")

## Load data

In [3]:
dataset = datasets.DiabetesDataset

np_x, np_y = dataset.get_dataset(balancing=True, discretizing=True, hot_encoding=True)
x_data, y_data = torch.Tensor(np_x), torch.Tensor(np_y)
input_size = x_data.size(1)
print(x_data.size())

torch.Size([536, 29])


In [4]:
def eval_model(x_train, y_train, x_valid, y_valid, model, metrics_average="binary"):
    model.eval()
    pred_train = model(x_train).detach()
    pred_valid = model(x_valid).detach()

    p_train, r_train, f_train, _ = metrics.precision_recall_fscore_support(y_train, pred_train, beta=1, average=metrics_average, labels=[0,1])
    p_valid, r_valid, f_valid, _ = metrics.precision_recall_fscore_support(y_valid, pred_valid, beta=1, average=metrics_average, labels=[0,1])

    return f_train, p_train, r_train, f_valid, p_valid, r_valid

def train_eval_model(x_train, y_train, x_valid, y_valid, model, criterion, optimizer, epochs=50, metrics_average="binary"):
    train_model(x_train, y_train, model, criterion, optimizer, epochs)
    return eval_model(x_train, y_train, x_valid, y_valid, model, metrics_average)

def print_eval(x_train, y_train, x_valid, y_valid, model, criterion, optimizer):
    f_train, p_train, r_train, f_valid, p_valid, r_valid = train_eval_model(x_train, y_train, x_valid, y_valid, model, criterion, optimizer)
    print(
        f"{'':<15}{'Train':^15}{'Valid':^15}",
        f"{'F1 score':<15}{f_train:^15.3f}{f_valid:^15.3f}",
        f"{'Precision':<15}{p_train:^15.3f}{p_valid:^15.3f}",
        f"{'Rappel':<15}{r_train:^15.3f}{r_valid:^15.3f}",
        sep="\n",
    )

## Network

In [5]:
class nApxNet(nn.Module):
    def __init__(self, n, hl=3, max_weighting="all") -> None:
        super().__init__()

        self.n_apx = n
        self.net = nn.Sequential(OrderedDict([
            ('nets', Parallel(OrderedDict([
                (f'apx{i}', ApproxNet(hl)) for i in range(1, self.n_apx+1)
            ]))),
            ('or_', MaxLayer(backward_method=max_weighting)),
        ]))

    def forward(self, input):
        return self.net(input)
    
    def forward_apx_only(self, input):
        return mtf.maximum([self.net.nets.get_submodule(f"apx{i}")(input) for i in range(1, self.n_apx+1)])
    
    def add_apx(self, module):
        self.n_apx += 1
        self.net.nets.add_module(f'apx{self.n_apx}', module)
    
    def add_nn(self, module):
        self.net.nets.add_module(f'nn', module)
    
class ApproxNet(nn.Module):
    def __init__(self, hl1):
        super().__init__()
        self.nn = nn.Sequential(OrderedDict([
            ('l1', nn.Linear(input_size,hl1)),
            ('a1', StepActivation()),
            ('l2', nn.Linear(hl1,5)),
            ('a2', StepActivation()),
            ('l3', nn.Linear(5,1)),
            ('a3', StepActivation()),
        ]))        

    def forward(self, x):
        x = self.nn(x)

        return x
    
class Net(nn.Module):
    def __init__(self):
        super().__init__()

        hl1 = 50
        hl2 = 25

        self. nn = nn.Sequential(OrderedDict([
            ('l1', nn.Linear(input_size,hl1)),
            ('a1', nn.ReLU()),
            ('l2', nn.Linear(hl1,hl2)),
            ('a2', nn.ReLU()),
            ('l3', nn.Linear(hl2,1)),
            ('a3', StepActivation()),
        ]))
    
    def forward(self, x):
        x = self.nn(x)

        return x

## Exp

In [6]:
from typing import Any


class NN_Model(Model):
    def forward(self, x):
        x = super().forward(x)

        if not self.training:
            x = x.round()
        
        return x

class RandomForest(RandomForestClassifier):
    def eval(self):
        pass
    
    def __call__(self, x):
        x = self.predict(x)
        return torch.Tensor(x)

def compute_metrics(model, x_valid, y_valid):
    model.eval()
    global_pred_valid = model(x_valid).detach()
    # apx_pred_valid = model.forward_apx_only(x_valid).detach()
    # nn_pred_valid = model.net.nets.nn(x_valid).detach()

    p_valid, r_valid, f_valid, _ = metrics.precision_recall_fscore_support(y_valid, global_pred_valid, beta=1, average=None, labels=[0,1], zero_division=0)

    return f_valid.mean(), p_valid[0], p_valid[1], r_valid[0], r_valid[1]

def nn_train(x_train, y_train, x_valid, y_valid, epochs=500):
    model = NN_Model(input_size)
    criterion = nn.BCELoss()
    optimizer = Adam(model.parameters(), lr=1e-2, weight_decay=1e-6)

    train_model(x_train, y_train, model, criterion, optimizer, epochs)

    return compute_metrics(model, x_valid, y_valid)

def rf_train(x_train, y_train, x_valid, y_valid):
    model = RandomForest()

    model.fit(x_train, y_train.reshape(-1))

    return compute_metrics(model, x_valid, y_valid)

def apx_train(x_train, y_train, x_valid, y_valid, epochs=500, n=3, p=1):   
    model = nApxNet(0, hl=5)
    criterion = AsymBCELoss(p) # HERE
    for _ in range(n):
        module = ApproxNet(5)
        optimizer = Adam(module.parameters(), lr=1e-2, weight_decay=1e-6)
        model.add_apx(module)

        train_model(x_train, y_train, model, criterion, optimizer, epochs)
        mtf.freeze_model(model)

    module = Net()
    model.add_nn(module)
    criterion = nn.BCELoss() # CHANGE ???
    optimizer = Adam(module.parameters(), lr=1e-2, weight_decay=1e-6)

    train_model(x_train, y_train, model, criterion, optimizer, epochs)

    return compute_metrics(model, x_valid, y_valid)

In [7]:
max_iter = 100
for train_func in [apx_train]:
    sum = np.zeros(5) 
    for _ in range(max_iter):
        train_index, valid_index = torch.utils.data.random_split(range(x_data.size(0)), [0.9, 0.1])

        x_train, y_train = x_data[train_index], y_data[train_index]
        x_valid, y_valid = x_data[valid_index], y_data[valid_index]

        res = train_func(x_train, y_train, x_valid, y_valid)
        res = np.array(res)
        sum+=res

    sum/=max_iter
    print(train_func.__name__.upper())
    print(f"{sum[0]:.2f} & {sum[1]:.2f} & {sum[2]:.2f} & {sum[3]:.2f} & {sum[4]:.2f}", end="\n\n")

APX_TRAIN
0.69 & 0.75 & 0.68 & 0.59 & 0.80

